<a href="https://colab.research.google.com/github/duygusezr/e-ticaret_urun_yorumlari/blob/main/e_ticaret_urun_yorumlari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import re
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df2 = pd.read_csv("e-ticaret_urun_yorumlari.csv",sep = ";")
df = df2.copy()
df.head(15)

,Metin,Durum
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1
5,ürün gerçekten çok güzel,1
6,Ürün beklediğimden güzel çıktı gayet kullanışl...,1
7,güzel makina tavsiye ederim,1
8,tavsiye edebileceğim çok güzel bir makina,1
9,ürün geldiğinde şarjı vardı. ilk lullanım önce...,1


In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

df["Metin"] = df["Metin"].apply(clean_text)

# Tokenizer ile kelimeleri sayısal değerlere çevirme
tokenizer = Tokenizer(num_words=4000)  # En sık kullanılan 5000 kelimeyi al
tokenizer.fit_on_texts(df["Metin"])
sequences = tokenizer.texts_to_sequences(df["Metin"])


max_length = 40
X = pad_sequences(sequences, maxlen=max_length)

# Etiketleri hazırlama (one-hot encoding)
y = to_categorical(df["Durum"], num_classes=3)  # 0, 1, 2 için 3 sınıf


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
vocab_size = 4000  # Tokenizer'daki kelime sayısı
embedding_dim = 100  # Her kelime için vektör boyutu

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(128, return_sequences=True))  # İlk LSTM katmanı
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))  # İkinci LSTM katmanı
model.add(Dropout(0.2))
model.add(LSTM(32))  # Üçüncü LSTM katmanı
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True  # En iyi ağırlıkları geri yükle
)

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test),
    #callbacks=[early_stopping]
)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.6429 - loss: 0.7573 - val_accuracy: 0.8784 - val_loss: 0.3381
Epoch 2/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8923 - loss: 0.2928 - val_accuracy: 0.8823 - val_loss: 0.3194
Epoch 3/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.9317 - loss: 0.1969 - val_accuracy: 0.8856 - val_loss: 0.3159
Epoch 4/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9443 - loss: 0.1638 - val_accuracy: 0.8876 - val_loss: 0.3356
Epoch 5/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9560 - loss: 0.1277 - val_accuracy: 0.8837 - val_loss: 0.3566
Epoch 6/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9660 - loss: 0.1078 - val_accuracy: 0.8800 - val_loss: 0.4032
Epoch 7/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.9719 - loss: 0.0989 - val_accuracy: 0.8827 - val_loss: 0.4054
Epoch 8/10
380/380 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9733 - loss: 0.0870 - val_acc

In [6]:
def predict_sentiment(text):
    cleaned_text = clean_text(text)
    sequence = tokenizer.texts_to_sequences([cleaned_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)
    prediction = model.predict(padded_sequence)
    sentiment = np.argmax(prediction)
    return sentiment  # 0: olumsuz, 1: olumlu, 2: nötr


sample_text = "harikulade"
if predict_sentiment(sample_text) == 0:
    print("Olumsuz")
elif predict_sentiment(sample_text) == 1:
    print("Olumlu")
else:
    print("Nötr")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Olumlu
